####Example 1: Cast
Is the movie good or bad based on the cast

Checking out the data..

In [55]:
!head -n 2 ex1.test.vw

1 Avengers_Age_of_Ultron| DowneyJr._Robert DowneyJr._Robert Hemsworth_Chris Ruffalo_Mark Ruffalo_Mark Evans_Chris Evans_Chris Johansson_Scarlett Johansson_Scarlett Renner_Jeremy Renner_Jeremy Spader_James Jackson_SamuelL. Cheadle_Don Cheadle_Don Taylor-Johnson_Aaron Taylor-Johnson_Aaron Olsen_Elizabeth Olsen_Elizabeth Bettany_Paul Bettany_Paul Smulders_Cobie Mackie_Anthony Mackie_Anthony Atwell_Hayley Elba_Idris Cardellini_Linda Skarsgård_Stellan Kim_Claudia Kretschmann_Thomas Serkis_Andy Lee_Stan Delpy_Julie Goodman_Henry Luca_Chris Schaeffer_Brian Provost-Chalkley_Dominique Andrews_Isaac Jones_GarethKieran WooLim_Chan Yeo_Minhee Kalu_Bentley Bleach_Julian Beasley_Chris Dabula_Vuyo Tembe_Nondumiso Thai_Kabelo Ledwaba_Lele Gaduka_Mandla Manamela_Harriet Hashe_Beulah Kumalo_Musca September_Mathapelo Acheampong_Antony Anyanwu_Chioma Deila_Ingvild Yeo_Sunny Go_Namju Kweon_Mina Kim_Earl Lee_Arthur Hewlitt_Verity Matovski_Michael Noce_Alma Richetta_Riccardo Ruff_Constanza Mclaren-Clark_Mont

In [63]:
!vw ex1.train.vw -f ex1.model

Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
final_regressor = ex1.model
using no cache
Reading datafile = ex1.train.vw
num sources = 1
average    since         example     example  current  current  current
loss       last          counter      weight    label  predict features
0.000000   0.000000            1         1.0   0.0000   0.0000       26
0.000000   0.000000            2         2.0   0.0000   0.0000        6
0.439921   0.879842            4         4.0   1.0000   0.1284       13
0.465951   0.491981            8         8.0   1.0000   0.2291        5
0.408777   0.351604           16        16.0   1.0000   0.3885       10
0.337837   0.266896           32        32.0   1.0000   0.4459        3
0.295468   0.253100           64        64.0   1.0000   0.4981       20
0.269744   0.244020          128       128.0   0.0000   0.5006       13
0.254644   0.239544          256       256.0   1.0000   0.5686        7
0.251240   0.247836          512       512.0   1

Now get the predictions on the test data. The -i argument specifies the input model file, the -t argument means "test only" and the -p argument specifies the output for the predictions. --quiet suppresses the other output from vw

In [64]:
!vw -i ex1.model -t ex1.test.vw -p /dev/stdout --quiet

1.000000 Avengers_Age_of_Ultron
0.010282 Barb_Wire
0.034936 Dude,_Where's_My_Car?
0.255498 Pulp_Fiction
1.000000 Schindler's_List
0.794812 Taken_3
0.145960 Terminator_2_Judgment_Day
0.361363 The_Human_Centipede_(First_Sequence)
1.000000 The_Lord_of_the_Rings_The_Fellowship_of_the_Ring
0.767750 The_Matrix_Reloaded
0.095027 Transformers_Age_of_Extinction


As we can see, this isn't all that good.. Let's try increasing the number of iterations

In [70]:
!vw ex1.train.vw -f ex1.model --passes 30 -c -k

Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
final_regressor = ex1.model
creating cache_file = ex1.train.vw.cache
Reading datafile = ex1.train.vw
num sources = 1
average    since         example     example  current  current  current
loss       last          counter      weight    label  predict features
0.000000   0.000000            1         1.0   0.0000   0.0000       26
0.000000   0.000000            2         2.0   0.0000   0.0000        6
0.439921   0.879842            4         4.0   1.0000   0.1284       13
0.465951   0.491981            8         8.0   1.0000   0.2291        5
0.410851   0.355751           16        16.0   0.0000   0.4095       12
0.332289   0.253727           32        32.0   0.0000   0.4389       12
0.294920   0.257551           64        64.0   0.0000   0.5056       10
0.269914   0.244907          128       128.0   1.0000   0.5198        7
0.255856   0.241799          256       256.0   0.0000   0.5456        

Get the predictions again

In [71]:
!vw -i ex1.model -t ex1.test.vw -p /dev/stdout --quiet

1.000000 Avengers_Age_of_Ultron
0.111372 Barb_Wire
0.026296 Dude,_Where's_My_Car?
0.229648 Pulp_Fiction
1.000000 Schindler's_List
0.664461 Taken_3
0.239987 Terminator_2_Judgment_Day
0.278806 The_Human_Centipede_(First_Sequence)
1.000000 The_Lord_of_the_Rings_The_Fellowship_of_the_Ring
0.697005 The_Matrix_Reloaded
0.280272 Transformers_Age_of_Extinction


In [72]:
def convert_to_labels(filename):
    with open(filename) as f:
        for line in f:
            parts = line.split(' ')
            label = 'Good' if float(parts[0]) >= 0.5 else 'Bad '
            print label, parts[1],

In [73]:
!vw -i ex1.model -t ex1.test.vw -p ex1.predictions --quiet

In [74]:
convert_to_labels('ex1.predictions')

Good Avengers_Age_of_Ultron
Bad  Barb_Wire
Bad  Dude,_Where's_My_Car?
Bad  Pulp_Fiction
Good Schindler's_List
Good Taken_3
Bad  Terminator_2_Judgment_Day
Bad  The_Human_Centipede_(First_Sequence)
Good The_Lord_of_the_Rings_The_Fellowship_of_the_Ring
Good The_Matrix_Reloaded
Bad  Transformers_Age_of_Extinction


Not looking so good.. let's filter out some cast noise and add directors

In [75]:
!head -n 3 ex2.test.vw

1 Avengers_Age_of_Ultron| DowneyJr._Robert DowneyJr._Robert Hemsworth_Chris Ruffalo_Mark Ruffalo_Mark director=Whedon_Joss
0 Barb_Wire| Aboulela_Amir Alexander_Adriana Andriole_David Asher_VanessaLee Balicki_Ron director=Hogan_David
0 Dude,_Where's_My_Car?| Kutcher_Ashton Scott_SeannWilliam Garner_Jennifer Sokoloff_Marla Swanson_Kristy director=Leiner_Danny


In [76]:
!vw ex2.train.vw -f ex2.model --passes 30 -c -k

Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
final_regressor = ex2.model
creating cache_file = ex2.train.vw.cache
Reading datafile = ex2.train.vw
num sources = 1
average    since         example     example  current  current  current
loss       last          counter      weight    label  predict features
1.000000   1.000000            1         1.0   1.0000   0.0000        2
0.532112   0.064223            2         2.0   0.0000   0.2534        7
0.427359   0.322606            4         4.0   1.0000   0.2345        2
0.384870   0.342380            8         8.0   1.0000   0.4063        7
0.322295   0.259721           16        16.0   1.0000   0.3567        7
0.294320   0.266346           32        32.0   0.0000   0.5418        7
0.267498   0.240675           64        64.0   1.0000   0.5901        2
0.257469   0.247440          128       128.0   0.0000   0.5164        7
0.246553   0.235637          256       256.0   0.0000   0.6037        

In [77]:
!vw -i ex2.model -t ex2.test.vw -p ex2.predictions --quiet
convert_to_labels('ex2.predictions')

Good Avengers_Age_of_Ultron
Bad  Barb_Wire
Bad  Dude,_Where's_My_Car?
Good Pulp_Fiction
Good Schindler's_List
Bad  Taken_3
Good Terminator_2_Judgment_Day
Bad  The_Human_Centipede_(First_Sequence)
Good The_Lord_of_the_Rings_The_Fellowship_of_the_Ring
Good The_Matrix_Reloaded
Good Transformers_Age_of_Extinction


What about multi-class labels?

In [7]:
!vw ex3.train.vw -f ex3.model --passes 30 --oaa 5 -c -k

Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
final_regressor = ex3.model
creating cache_file = ex3.train.vw.cache
Reading datafile = ex3.train.vw
num sources = 1
average    since         example     example  current  current  current
loss       last          counter      weight    label  predict features
1.000000   1.000000          1      1.0          4        1        2
1.000000   1.000000          2      2.0          3        4        7
0.750000   0.500000          4      4.0          4        4        2
0.625000   0.500000          8      8.0          4        4        7
0.687500   0.750000         16     16.0          5        3        7
0.625000   0.562500         32     32.0          3        4        7
0.562500   0.500000         64     64.0          4        4        2
0.515625   0.468750        128    128.0          3        4        7
0.500000   0.484375        256    256.0          3        4        4
0.519531   0.539062      

In [8]:
!vw -i ex3.model -t ex3.test.vw -p /dev/stdout --quiet

4.000000 Avengers_Age_of_Ultron
3.000000 Barb_Wire
3.000000 Dude,_Where's_My_Car?
4.000000 Pulp_Fiction
4.000000 Schindler's_List
3.000000 Taken_3
4.000000 Terminator_2_Judgment_Day
3.000000 The_Human_Centipede_(First_Sequence)
3.000000 The_Lord_of_the_Rings_The_Fellowship_of_the_Ring
4.000000 The_Matrix_Reloaded
3.000000 Transformers_Age_of_Extinction


In [9]:
!cat ex3.test.vw

4 Avengers_Age_of_Ultron| DowneyJr._Robert DowneyJr._Robert Hemsworth_Chris Ruffalo_Mark Ruffalo_Mark director=Whedon_Joss
2 Barb_Wire| Aboulela_Amir Alexander_Adriana Andriole_David Asher_VanessaLee Balicki_Ron director=Hogan_David
3 Dude,_Where's_My_Car?| Kutcher_Ashton Scott_SeannWilliam Garner_Jennifer Sokoloff_Marla Swanson_Kristy director=Leiner_Danny
5 Pulp_Fiction| Roth_Tim Roth_Tim Plummer_Amanda Plummer_Amanda Lovelace_Laura director=Tarantino_Quentin
5 Schindler's_List| Neeson_Liam Kingsley_Ben Fiennes_Ralph Goodall_Caroline Sagall_Jonathan director=Spielberg_Steven
4 Taken_3| Neeson_Liam Whitaker_Forest Janssen_Famke Grace_Maggie Scott_Dougray director=Megaton_Olivier
5 Terminator_2_Judgment_Day| Schwarzenegger_Arnold Hamilton_Linda Furlong_Edward Patrick_Robert Boen_Earl director=Cameron_James
3 The_Human_Centipede_(First_Sequence)| Laser_Dieter Williams_AshleyC. Yennie_Ashlynn Kitamura_Akihiro Leupold_Andreas director=Six_Tom
5 The_Lord_of_the_Rings_The_Fellowship_of_the_